In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)



In [3]:
wrr_df = mss_df.loc[mss_df['UniqueID'].str.contains("WRR")].copy()

In [13]:
client_df = wrr_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


# Normalize species

species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))

# Normalize Origin

origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))



In [14]:
"""
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
client_df['id'] = client_df['uniqueid']
client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]
"""

"\nclient_df.columns = [x.lower() for x in client_df.columns]\nclient_df.columns = [x.replace(' ', '_') for x in client_df.columns]\nclient_df['id'] = client_df['uniqueid']\nclient_df.drop(columns=['uniqueid'],inplace=True)\nclient_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',\n                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]\n"

In [15]:
ga_intake_df = wrr_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [17]:
status_df = wrr_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [18]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [28]:
wrr_inprog = wrr_df[wrr_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
wrr_inprog['Tract'] = wrr_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['CMP'] = wrr_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['Stand'] = wrr_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['TCS'] = wrr_inprog['Tract'] + "_" + wrr_inprog['CMP'] + "_" + wrr_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = wrr_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = wrr_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)

# looking for TCS combinations in previous ticketids        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
if len(ticket_id_dict) > 0:
    for k, v in ticket_id_dict.items():
        trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k
else:
    trial_df['otherid'] = None
    

def final_id(row):
    if row['otherid'] != None:
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(wrr_df['UniqueID'], wrr_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WAR"), 'TicketID'] = new_tcs_intake['old_ticketids']


ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

In [30]:
ticketid_table[ticketid_table['TicketID'].str.contains("X")]

,TicketID,UniqueID
14,WRR240101CCXX,WRR240101CC01
15,WRR240101CCXX,WRR240101CC02
16,WRR240101CCXX,WRR240101CC03
132,WRR210317XXXX,WRR210317AH24
133,WRR210317XXXX,WRR210317TH01
134,WRR210317XXXX,WRR210317TH02
135,WRR210317XXXX,WRR210317TH03
136,WRR210317XXXX,WRR210317TH04
137,WRR210317XXXX,WRR210317TH05
138,WRR210317XXXX,WRR210317TH06


Getting the mill tickets

In [39]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WRR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WRR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

# big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
# big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
# big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
# big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'], errors='coerce')

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WRR_RP5
WRR_RP5_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP6
WRR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP7
WRR_RP7_Compiled.xlsm
P:\Mill Tickets\WRR_RP8
WRR_RP8_Compiled.xlsm


In [40]:
mill_df

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week
0,2021-07-06,875937,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.27,0.000,23.270,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
1,2021-07-06,875841,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.86,0.000,23.860,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
2,2021-07-06,875787,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.99,0.000,23.990,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
3,2021-07-06,876023,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.16,0.000,28.160,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
4,2021-07-06,876026,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.04,0.000,28.040,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
280,2024-08-01,300140,Georgia-Pacific - Cedar Springs Paper Mill,PPW,27.47,0.000,27.470,W6_Updated_Mayhaw 122-1 ST 1 WE 08.03.24.1.pdf,NaN,8.0,WRR240404TH02,Week05
281,2024-08-01,300165,Georgia-Pacific - Cedar Springs Paper Mill,PPW,29.26,0.000,29.260,W6_Updated_Mayhaw 122-1 ST 1 WE 08.03.24.1.pdf,NaN,8.0,WRR240404TH02,Week05
282,2024-07-29,141578,Georgia-Pacific - Albany Lumber,CNS,25.40,0.000,25.400,W6_Updated_Mayhaw 122-1 ST 1 WE 08.03.24.1.pdf,NaN,8.0,WRR240404TH02,Week05
283,2024-08-01,141866,Georgia-Pacific - Albany Lumber,CNS,25.47,0.228,25.242,W6_Updated_Mayhaw 122-1 ST 1 WE 08.03.24.1.pdf,NaN,8.0,WRR240404TH02,Week05


In [41]:
# Getting the old Harvest TicketIDs
harvested_tons_oldTID = mill_df[mill_df['Reporting Period'] == 7].groupby("TicketID")['Tons (U.S)'].sum().reset_index()

In [42]:
harvested_tons_oldTID['Tons (U.S)'].sum()

63343.458

Let's get new and old TIDs that are currently in process

In [43]:
# Getting harvest ids that were in RP 7 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [44]:
# Let's see how many tons are under old TIDs of activities in question
harvested_tons_oldTID[harvested_tons_oldTID['TicketID'].isin(harvest_old_ticketid)]['Tons (U.S)'].sum()

4989.77

In [45]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [46]:
# Let's see what is in old_ticketid that is not in new_ticketIDs
[item for item in harvest_old_ticketid if item not in new_ticketIDs]

[]

In [47]:
for id in harvest_old_ticketid:
    old_unique = wrr_df[wrr_df['TicketID'] == id]['UniqueID'].unique().tolist()
    new_unique = ticketid_table[ticketid_table['TicketID'] == id]['UniqueID'].unique().tolist()
    inold_notinew = [item for item in old_unique if item not in new_unique]
    if len(inold_notinew) > 0:
        print(id, old_unique)

In [53]:
trial_expected = pd.merge(ticketid_table, wrr_df[['UniqueID', 'Current RP Expected GT', 'Status', 'Future GT']], on='UniqueID', how='left')
trial_expected = pd.merge(trial_expected, client_df[['UniqueID', 'Reporting Period']], on='UniqueID', how='left')
wrr_rp7_expected = trial_expected[(trial_expected['Reporting Period'] == 7) & trial_expected['Status'].isin(['IN PROCESS', 'OPEN', 'POST'])].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
wrr_rp7_expected = pd.merge(wrr_rp7_expected, harvested_tons_oldTID, on='TicketID', how='left')
wrr_rp7_expected['Tons (U.S)'] = pd.to_numeric(wrr_rp7_expected['Tons (U.S)'], errors='coerce', downcast='float')
wrr_rp7_expected['Current RP Expected GT'] = pd.to_numeric(wrr_rp7_expected['Current RP Expected GT'], errors='coerce', downcast='float')
wrr_rp7_expected['Tons (U.S)'].fillna(0, inplace=True)
wrr_rp7_expected['Current RP Expected GT'].fillna(0, inplace=True)
wrr_rp7_expected['Remaining'] = wrr_rp7_expected['Current RP Expected GT'] - wrr_rp7_expected['Tons (U.S)']

In [55]:
wrr_rp7_expected[wrr_rp7_expected['Remaining'] > 0]

,TicketID,Current RP Expected GT,Tons (U.S),Remaining
0,WRR230829CC04,252.679993,0.000000,252.679993
1,WRR231107CC02,9338.919922,4852.720215,4486.199707
2,WRR231107TT01,1648.609985,137.050003,1511.559937
8,WRR240101CCXX,12298.480469,0.000000,12298.480469
9,WRR240101TH01,541.820007,0.000000,541.820007
10,WRR240101TH02,888.250000,0.000000,888.250000
11,WRR240101TT01,999.900024,0.000000,999.900024
12,WRR240404TH01,920.400024,0.000000,920.400024
13,WRR240404TH02,2090.399902,0.000000,2090.399902
14,WRR240517TH01,5916.000000,0.000000,5916.000000


In [56]:
wrr_rp7_expected.to_excel("wrr_rp7_accouting.xlsx")